Домашнее задание №2. Тема "BM25"

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks')

In [0]:
import pandas as pd
import numpy as np

In [0]:
table = pd.read_csv('quora_question_pairs_rus.csv')

In [7]:
table.head()

,Unnamed: 0,question1,question2,is_duplicate
0,0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1


In [0]:
N = len(table)

In [0]:
test_N = 10000

In [0]:
test_table = table.head(test_N)

In [0]:
doc_lens = []
for row in test_table['question2']:
  doc_lens.append(len(str(row).split()))

In [0]:
len_mean = np.mean(doc_lens)

In [14]:
pip install pymorphy2

In [0]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [0]:
texts = []
for row in test_table['question2']:
  texts.append(str(row))

In [0]:
import re
corpus = []#массив массивов лемм слов всех текстов
for t in texts:#for text
    ws = []
    if t != "":
      words = t.split()
      for w in words:
        w = re.sub('[.,-;:?!@#$%^&()_+=—\ufeff–"…«»>wwwtvsubtitlesnet]', '', w).lower()#чистим слова
        if w != "":
          p = morph.parse(w)[0]
          if p not in ws:
            ws.append(p.normal_form)
      corpus.append(ws)

In [0]:
queries = []
for row in test_table['question1']:
  queries.append(str(row))

In [0]:
import re
query_corpus = []#массив массивов лемм слов всех текстов запросов
for q in queries:#for text
    ws = []
    if q != "":
      words = q.split()
      for w in words:
        w = re.sub('[.,-;:?!@#$%^&()_+=—\ufeff–"…«»>wwwtvsubtitlesnet]', '', w).lower()#чистим слова
        if w != "":
          p = morph.parse(w)[0]
          if p not in ws:
            ws.append(p.normal_form)
      query_corpus.append(ws)

In [0]:
def tokenize(doc):
  ws = []
  words = doc.split()
  for w in words:
    w = re.sub('[.,-;:?!@#$%^&()_+=—\ufeff–"…«»>wwwtvsubtitlesnet]', '', w).lower()
    if w != "":
      p = morph.parse(w)[0]
      ws.append(p.normal_form)
  return ws

способ первый с матрицей

In [42]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
stopwords = set(nltk.corpus.stopwords.words('russian')) #converting stopwords to a set for faster processing in the future.
stemmer = nltk.stem.PorterStemmer() 

#Function to extract and tokenize terms from a document
def extract_and_tokenize_terms(doc):
    terms = []
    for token in tokenize(doc):
        if token not in stopwords: # 'in' and 'not in' operations are faster over sets than lists
            if not re.search(r'\d',token) and not re.search(r'[^A-Za-z-]',token): #Removing numbers and punctuations 
                #(excluding hyphenated words)
                terms.append(stemmer.stem(token.lower()))
    return terms

In [0]:
documents = {}

In [0]:
for i in range(len(corpus)):
  documents[str(i)] = corpus[i]

In [46]:
documents['0']

['что',
 'произойти',
 'если',
 'правительство',
 'индия',
 'украсть',
 'кохинор',
 'кохинооралмаз',
 'назад']

In [0]:
#Building an inverted index for the documents

from collections import defaultdict
    
inverted_index = defaultdict(set)

for docid, terms in documents.items():
    for term in terms:
        inverted_index[term].add(docid)   

In [0]:
#Building a TF-IDF representation using BM25 

NO_DOCS = len(documents) #Number of documents

AVG_LEN_DOC = sum([len(doc) for doc in documents.values()])/len(documents) #Average length of documents

#The function below takes the documentid, and the term, to calculate scores for the tf and idf
#components, and multiplies them together.
def tf_idf_score(k1,b,term,docid):  
    
    ft = len(inverted_index[term]) 
    term = stemmer.stem(term.lower())
    fdt =  documents[docid].count(term)
    
    idf_comp = math.log((NO_DOCS - ft + 0.5)/(ft+0.5))
    
    tf_comp = ((k1 + 1)*fdt)/(k1*((1-b) + b*(len(documents[docid])/AVG_LEN_DOC))+fdt)
    
    return idf_comp * tf_comp

#Function to create tf_idf matrix without the query component
def create_tf_idf(k1,b):
    tf_idf = defaultdict(dict)
    for term in set(inverted_index.keys()):
        for docid in inverted_index[term]:
            tf_idf[term][docid] = tf_idf_score(k1,b,term,docid)
    return tf_idf

In [0]:
#Creating tf_idf matrix with said parameter values: k1 and b for all documents.
import math
from math import log
tf_idf = create_tf_idf(2,0.75)

In [0]:
#Function to retrieve query component
def get_qtf_comp(k3,term,fqt):
    return ((k3+1)*fqt[term])/(k3 + fqt[term])


#Function to retrieve documents || Returns a set of documents and their relevance scores. 
def retr_docs(query,result_count):
    q_terms = [stemmer.stem(term.lower()) for term in query.split() if term not in stopwords] #Removing stopwords from queries
    fqt = {}
    for term in q_terms:
        fqt[term] = fqt.get(term,0) + 1
    
    scores = {}
    
    for word in fqt.keys():
        #print word + ': '+ str(inverted_index[word])
        for document in inverted_index[word]:
            scores[document] = scores.get(document,0) + (tf_idf[word][document]*get_qtf_comp(0,word,fqt)) #k3 chosen as 0 (default)
    
    return sorted(scores.items(),key = lambda x : x[1] , reverse=True)[:result_count] 

In [53]:
retr_docs("история кохинор",5)

[('0', 8.89463830521871),
 ('8236', 7.768555349733469),
 ('3955', 7.179381230455969),
 ('7273', 7.179381230455969),
 ('1165', 6.673274203530167)]

Еще две функции для расчета с матрицей и без

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

In [0]:
arrstr = []
for text in corpus:  
  s = ' '.join(text)
  arrstr.append(s)

In [0]:
X = vectorizer.fit_transform(arrstr)
matrix = X.toarray()

In [0]:
words = vectorizer.get_feature_names()

In [0]:
tf_matrix = matrix / np.array(doc_lens).reshape((-1, 1)) #матрица с обычными значениями tf

In [0]:
def tf_matrix_func(tf_matrix, b): 
    k = 2
    pairs =  np.ndenumerate(tf_matrix)
    for i, value in pairs:
        doc_id = i[0]
        l = doc_lens[doc_id]
        tf_matrix[i] = (value * (k + 1.0)) / (value + k * (1.0 - b + b * (l/len_mean)))
    return tf_matrix

In [0]:
from math import log
num_doc_qi = np.count_nonzero(matrix, axis=0) #number of docs with qi

def idf_score(word):
    word_id = words.index(word)
    num = num_doc_qi[word_id]
    score = log((test_N - num + 0.5) / (num + 0.5))
    return score

In [61]:
idf_score('кохинор')

8.804825262617976

In [0]:
all_idfs = []
for word in words:
  score = idf_score(word)
  all_idfs.append(score)

In [0]:
def tokenize(query):
  ws = []
  query_words = query.split()
  for w in query_words:
    w = re.sub('[.,-;:?!@#$%^&()_+=—\ufeff–"…«»>wwwtvsubtitlesnet]', '', w).lower()
    if w != "":
      p = morph.parse(w)[0]
      ws.append(p.normal_form)
  return ws

In [0]:
b = 0.75

In [0]:
tf_matrix_upd_1 = tf_matrix_func(tf_matrix, b) #матрица со значениями по новой формуле

In [0]:
for i in range(len(tf_matrix_upd_1)):
  for j in range(len(tf_matrix_upd_1[i])):
    tf_matrix_upd_1[i][j] = tf_matrix_upd_1[i][j] * all_idfs[j] #матрица со значениями bm25 для корпуса

In [0]:
def bm25_vect_ver(query, b):
  query_tokenized = tokenize(query)
  new_query = [' '.join(query_tokenized)]
  query_vector = np.array(vectorizer.transform(new_query).todense())[0]
  query_idfs = np.array(all_idfs)*query_vector
  tf_matrix_new = tf_matrix_func(tf_matrix, b)
  score_bm25 = np.dot(tf_matrix_new,query_idfs)
  
  return score

In [0]:
def bm25_pair_ver(query, b):
  k = 2
  query_tokenized = tokenize(query)
  results = []
  for i in range(test_N):
    bm25_score = 0
    for j in range(len(query_tokenized)):
      word = query_tokenized[j]
      l = doc_lens[i]
      if word in words:
        word_id = words.index(word)
        tf_val = tf_matrix[(i, word_id)]
        tf_new = (tf_val * (k + 1.0))/(tf_val + k * (1.0 - b + b * (l/len_mean)))
        bm25_score += all_idfs[word_id] * tf_new
      else:
        bm25_score += 0
    results.append(bm25_score)
  return results

Сравнение времени работы последних двух функций

In [69]:
query_0 = 'история кохинор продолжается'
from time import time
start1 = time()
result_1 = bm25_vect_ver(query_0, b)
time1 = str(time() - start1)
print('Time of the first function: ' + time1)

start2 = time()
result_2 = bm25_pair_ver(query_0, b)
time2 = str(time() - start2)
print('Time of the second function: ' + time2)


Time of the first function: 177.41529178619385
Time of the second function: 3.844017267227173


Пример топ 10 со значениями bm25

In [0]:
query_2 = 'рождественские каникулы'

In [0]:
bm25_scores = bm25_pair_ver(query_2, b)

In [0]:
new_mas = []
new_list = sorted(enumerate(bm25_scores), key=lambda x:x[1], reverse=True)
for i in new_list:
  new_mas.append(i[0])

In [73]:
top_10 = new_mas[:10]
for rank, idx in enumerate(top_10):
  print('rank: ', rank)
  print('text: ', np.array(texts)[idx])
  print('bm25_score: ', bm25_scores[idx], '\n')

rank:  0
text:  каков ваш рождественский список
bm25_score:  16.869321934978363 

rank:  1
text:  я не хочу покупать маме рождественский подарок, что я должен делать
bm25_score:  7.110062994842695 

rank:  2
text:  что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад
bm25_score:  0.0 

rank:  3
text:  как повысить скорость интернета путем взлома через dns
bm25_score:  0.0 

rank:  4
text:  найти остаток, когда математика 23 ^ 24 математика разделена на 24 23
bm25_score:  0.0 

rank:  5
text:  какая рыба выживет в соленой воде
bm25_score:  0.0 

rank:  6
text:  Я тройная луна-козерог и восхождение в козероге, что это говорит обо мне
bm25_score:  0.0 

rank:  7
text:  что делает детей активными и далеки от телефонных и видеоигр
bm25_score:  0.0 

rank:  8
text:  что я должен делать, чтобы быть великим геологом?
bm25_score:  0.0 

rank:  9
text:  когда вы используете вместо
bm25_score:  0.0 



Приверим результаты с помощью питоновкого варианта подсчета bm_25

In [74]:
pip install rank_bm25

  Created wheel for rank-bm25: filename=rank_bm25-0.2-cp36-none-any.whl size=4162 sha256=076b2c2cb7c3c5b90c25e9d6fcfb96b52f9e55cf167f2f1aa88899b404476ad6
  Stored in directory: /root/.cache/pip/wheels/6f/0c/1f/78945dd6a5478bbcdb50d73ac96ae5af2ffcdfcd374fd9b1bf
Successfully built rank-bm25


In [0]:
from rank_bm25 import BM25Okapi

In [0]:
bm25 = BM25Okapi(corpus)

In [0]:
query_tokenized = tokenize(query_2)

In [78]:
query_tokenized

['рождественский', 'каникулы']

In [0]:
doc_scores = bm25.get_scores(query_tokenized)

In [80]:
r = 0
for i in bm25.get_top_n(query_tokenized, corpus, n=10):
  print("rank", r)
  r += 1
  print(i)

rank 0
['каков', 'ваш', 'рождественский', 'список']
rank 1
['я', 'не', 'хотеть', 'покупать', 'мама', 'рождественский', 'подарок', 'что', 'я', 'должный', 'делать']
rank 2
['безопасно', 'ли', 'для', 'женщина', 'путешествовать', 'в', 'одиночка', 'в', 'япония']
rank 3
['какой', 'странный', 'сувенир', 'для', 'покемон', 'в', 'луч', 'солнце', 'и', 'луна']
rank 4
['быть', 'работать', 'карта', 'r', 'на', 'do', 'd', 'x']
rank 5
['сколько', 'деньга', 'быть', 'составлять', 'рупия', 'за', 'год', 'на', 'годовой']
rank 6
['как', 'увеличить', 'рост']
rank 7
['античный', 'бизнес', 'юридический', 'в', 'индия']
rank 8
['который', 'являться', 'самый', 'известный', 'поэт']
rank 9
['что', 'вызывать', 'рост', 'продажа', 'pho', 'за', 'апрель', 'год']


Попробуем посчитать точность поиска

In [0]:
query_test_acc = queries[0]

In [89]:
query_test_acc

'Какова история кохинор кох-и-ноор-бриллиант'

In [0]:
bm25_scores_1 = bm25_pair_ver(query_test_acc, 0.75)

In [0]:
def test(query, b):
  bm25_scores = bm25_pair_ver(query, b)
  mas = []
  new_list = sorted(enumerate(bm25_scores), key=lambda x:x[1], reverse=True)
  for i in new_list:
    mas.append(i[0])
  top_5 = mas[:5]
  for rank, idx in enumerate(top_5):
    print('rank: ', rank)
    print('text: ', np.array(test_table)[idx])
    print('bm25_score: ', bm25_scores[idx], '\n')

In [86]:
top_5_1 = test(query_test_acc, 0.75)

rank:  0
text:  [8236 'прослеживает следующий napster' 'какова история напстера' 0]
bm25_score:  15.174031330553452 

rank:  1
text:  [7273 'каковы ваши истории неудач' 'какова ваша история неудачи' 1]
bm25_score:  13.714579692014158 

rank:  2
text:  [0 'Какова история кохинор кох-и-ноор-бриллиант'
 'что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад'
 0]
bm25_score:  10.517678980440579 

rank:  3
text:  [4447 'Каков ваш обзор сексуальных историй'
 'Каков ваш обзор истории Чу Юань' 0]
bm25_score:  10.499097402186198 

rank:  4
text:  [3955
 'делает работу s jio 4g сетевой работой на островах andaman и nicobar, особенно в своем столичном порту blair'
 'автомобиль история первый машина' 0]
bm25_score:  10.080596197228791 



In [87]:
top_5_2 = test(query_test_acc, 0)

rank:  0
text:  [8236 'прослеживает следующий napster' 'какова история напстера' 0]
bm25_score:  11.516692410969739 

rank:  1
text:  [7273 'каковы ваши истории неудач' 'какова ваша история неудачи' 1]
bm25_score:  10.713840670287311 

rank:  2
text:  [0 'Какова история кохинор кох-и-ноор-бриллиант'
 'что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад'
 0]
bm25_score:  10.30207068745296 

rank:  3
text:  [4447 'Каков ваш обзор сексуальных историй'
 'Каков ваш обзор истории Чу Юань' 0]
bm25_score:  8.812674806643008 

rank:  4
text:  [3955
 'делает работу s jio 4g сетевой работой на островах andaman и nicobar, особенно в своем столичном порту blair'
 'автомобиль история первый машина' 0]
bm25_score:  7.974068334408734 



In [88]:
top_5_2 = test(query_test_acc, 1)

rank:  0
text:  [8236 'прослеживает следующий napster' 'какова история напстера' 0]
bm25_score:  16.974967813052565 

rank:  1
text:  [7273 'каковы ваши истории неудач' 'какова ваша история неудачи' 1]
bm25_score:  15.1315828401832 

rank:  2
text:  [4447 'Каков ваш обзор сексуальных историй'
 'Каков ваш обзор истории Чу Юань' 0]
bm25_score:  11.216402507438065 

rank:  3
text:  [3955
 'делает работу s jio 4g сетевой работой на островах andaman и nicobar, особенно в своем столичном порту blair'
 'автомобиль история первый машина' 0]
bm25_score:  11.053981374623522 

rank:  4
text:  [0 'Какова история кохинор кох-и-ноор-бриллиант'
 'что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад'
 0]
bm25_score:  10.591568006156901 



В каждом варианте был найден пример, с которым у запроса значение is_duplicate = 1